# GANSFORMER
---------------
This notebook needs to be ran inside of the Tensorflow 1.15 Py3 GPU Docker container.

## Docker Configuration
If running this container for the first time, the lines in the cell below must be executed. We install requirements, install certain required Linux packages from APT, and define environment variables needed to compile the custom CUDA ops.

In [2]:
# # Prep Tensorflow Docker with required software and libraries
# !apt-get update
# !apt-get install ffmpeg libsm6 libxext6 -y
# !pip install -r requirements.txt
# !pip install joblib 

# # Set ENV variables for compiling TF ops
# !export PATH=/usr/local/cuda-10.0/bin${PATH:+:${PATH}}
# !export LD_LIBRARY_PATH=/usr/local/cuda-10.0/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}

In [2]:
!python3 --version

Python 3.6.9


In [3]:
!nvidia-smi

Mon Dec  6 16:56:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:09:00.0  On |                  N/A |
| 40%   34C    P8    22W / 260W |    951MiB / 10985MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Verify lung data volume mounted in container
!ls data | wc -l

194921


In [4]:
import os
import shutil
from PIL import Image
from joblib import Parallel, delayed
import cv2

---------------
## Build COVIDx Dataset
To keep things simple, we will utilize the tools provided by the team that developed gansformer. Here, we need to create a dataset so we have TF Records for the covidx image data.

Notes:
- Had to update `dnnlib/tflib/custom_ops.py` to account for TF 1.15 and allow multi cpu.
- Modify `dataset_tool.py` to capture image shape errors.
- Process all images so they are the same size, as the documentation is misleading.

In [9]:
# # Related to update of dnnlib
# !rm -rf /external_code/gan/gansformer/dnnlib/tflib/cudacache/

With the changes, we need to verify that the container is correctly configured and ready to compile the CUDA functions. `test_nvcc.cu` was pulled from the StyleGAN2 repo, as it was not available within the Gansformer library.

In [4]:
!nvcc test_nvcc.cu -o test_nvcc -run

CPU says hello.
GPU says hello.


#### Comments
There were issues creating a custom dataset using the provided tools. After opening up an issue on Github, I was informed that `prepare_data.py` sorts the files alphabetically. Thus, when I was trying to catch malformed images, I was actually moving ones that weren't causing the errors. I modified the the assert statment so that it would provided information on file issues. Here we see that there is a file of size 1024x1024. The data object stores the first image size as `self.shape` and expects all following images to be the same size.

In [7]:
!python prepare_data.py \
--task covidx \
--images-dir 'data' \
--format png --ratio 0.7 \
--shards-num 20 --max-images 194921

Preparing the covidx dataset...
Loading images from data
  8%|██▋                               | 15340/194921 [12:45<2:29:15, 20.05it/s]
Traceback (most recent call last):
  File "prepare_data.py", line 217, in <module>
    run_cmdline(sys.argv)
  File "prepare_data.py", line 214, in run_cmdline
    prepare(**vars(args))
  File "prepare_data.py", line 185, in prepare
    shards_num = shards_num, max_imgs = max_images)
  File "prepare_data.py", line 78, in <lambda>
    "png": lambda tfdir, imgdir, **kwargs: dataset_tool.create_from_imgs(tfdir, imgdir, format = "png", **kwargs),
  File "/tf/notebooks/Final Project/gansformer/dataset_tool.py", line 696, in create_from_imgs
    tfr.add_img(img)
  File "/tf/notebooks/Final Project/gansformer/dataset_tool.py", line 84, in add_img
    assert img.shape == self.shape, f'Img: {img.shape} Self: {self.shape} \n {img}'
AssertionError: Img: (3, 1024, 1024) Self: (3, 512, 512) 
 [[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 

In [ ]:
def resize_img(path):
    """
    Resize the images to 512x512 so the TF Record can be compiled.
    """
    temp_img = cv2.imread(path)
    temp_img = cv2.resize(temp_img, (512, 512))
    cv2.imwrite(path, temp_img)
        
# Process all images    
files = [os.path.join('data', x) for x in os.listdir('data')]

Parallel(n_jobs=24)(delayed(resize_img)(path) for path in files)

## DATASET IS BUILT DO NOT RERUN

STOP!

In [24]:
!python prepare_data.py \
--task covidx \
--images-dir 'data' \
--format png --ratio 0.7 \
--shards-num 20 --max-images 194921

Preparing the covidx dataset...
Loading images from data
100%|█████████████████████████████████| 194921/194921 [2:56:18<00:00, 18.43it/s]
Completed preparations for covidx!


------------------------------
# Train GANSFORMER
Starting with the clever dataset, we will continue training on our covid lung dataset. The hope is to transition this model to a new domain, without having to train for over a week using 8 or so GPUs. As the model trains, output will be generated in the `/gansformer/results/clever-*` directory. This includes model weights saved as a pickle file, as well has generated images and attention maps.

In [10]:
!python run_network.py \
--train \
--gpus 0 \
--ganformer-default \
--expname clevr-scratch \
--dataset covidx \
--eval-images-num 10000 \
--keep-samples True \
--metrics fid

Start model training from scratch
Local submit - run_dir: results/clevr-scratch-000
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset datasets...
Dataset shape:  [3, 256, 256]
Dynamic range:  [0, 255]
Constructing networks...
Setting up TensorFlow plugin 'fused_bias_act.cu': Preprocessing... Compiling... Loading... Done.
Setting up TensorFlow plugin 'upfirdn_2d.cu': Preprocessing... Compiling... Loading... Done.

G                                            Params    OutputShape         WeightShape     
---                                          ---       ---                 ---             
ltnt_emb/emb                                 512       (16, 32)            (16, 32)        
G_mapping/AttLayer_0                         6336      (?, 32)             -               
G_mapping/Dense0_0                           1056      (?, 32)             (32, 32)        
G_mapping/Dense0_1                           105


D                   Params    OutputShape         WeightShape     
---                 ---       ---                 ---             
ltnt_emb/emb        512       (16, 32)            (16, 32)        
256x256/FromRGB     512       (?, 128, 256, 256)  (1, 1, 3, 128)  
256x256/Conv0       147584    (?, 128, 256, 256)  (3, 3, 128, 128)
256x256/Conv1_down  295168    (?, 256, 128, 128)  (3, 3, 128, 256)
256x256/Skip        32768     (?, 256, 128, 128)  (1, 1, 128, 256)
128x128/Conv0       590080    (?, 256, 128, 128)  (3, 3, 256, 256)
128x128/Conv1_down  1180160   (?, 512, 64, 64)    (3, 3, 256, 512)
128x128/Skip        131072    (?, 512, 64, 64)    (1, 1, 256, 512)
64x64/Conv0         2359808   (?, 512, 64, 64)    (3, 3, 512, 512)
64x64/Conv1_down    2359808   (?, 512, 32, 32)    (3, 3, 512, 512)
64x64/Skip          262144    (?, 512, 32, 32)    (1, 1, 512, 512)
32x32/Conv0         2359808   (?, 512, 32, 32)    (3, 3, 512, 512)
32x32/Conv1_down    2359808   (?, 512, 16, 16)    (3, 3, 512,

tick 20    kimg    161.4   loss/reg: G ( 1.289  0.000) D ( 1.106  0.084)   grad norms: G ( 4.960  0.000) D ( 4.468  0.422)   time 9h 19m 11s sec/kimg  195.81 maxGPU  7.4GB clevr-scratch 
tick 21    kimg    169.5   loss/reg: G ( 1.260  0.000) D ( 1.073  0.086)   grad norms: G ( 5.172  0.000) D ( 4.133  0.437)   time 9h 45m 27s sec/kimg  195.48 maxGPU  7.4GB clevr-scratch 
network-snapshot-000169        time 5m 19s       fid 95.8610   
tick 22    kimg    177.5   loss/reg: G ( 1.310  0.000) D ( 1.060  0.095)   grad norms: G ( 6.001  0.000) D ( 3.942  0.458)   time 10h 17m 12s sec/kimg  195.48 maxGPU  7.4GB clevr-scratch 
tick 23    kimg    185.6   loss/reg: G ( 1.316  0.000) D ( 1.031  0.093)   grad norms: G ( 6.168  0.000) D ( 3.644  0.466)   time 10h 43m 28s sec/kimg  195.46 maxGPU  7.4GB clevr-scratch 
tick 24    kimg    193.7   loss/reg: G ( 1.244  0.000) D ( 1.049  0.085)   grad norms: G ( 5.491  0.000) D ( 3.301  0.472)   time 11h 09m 14s sec/kimg  191.72 maxGPU  7.4GB clevr-scratch

network-snapshot-000387        time 5m 19s       fid 47.2564   
tick 49    kimg    395.3   loss/reg: G ( 1.068  0.000) D ( 1.100  0.082)   grad norms: G ( 5.602  0.000) D ( 2.488  0.615)   time 22h 46m 14s sec/kimg  193.18 maxGPU  7.4GB clevr-scratch 
tick 50    kimg    403.3   loss/reg: G ( 1.079  0.000) D ( 1.078  0.089)   grad norms: G ( 6.001  0.000) D ( 2.541  0.615)   time 23h 12m 10s sec/kimg  192.97 maxGPU  7.4GB clevr-scratch 
tick 51    kimg    411.4   loss/reg: G ( 1.051  0.000) D ( 1.081  0.085)   grad norms: G ( 6.271  0.000) D ( 2.499  0.605)   time 23h 36m 14s sec/kimg  179.05 maxGPU  7.4GB clevr-scratch 
network-snapshot-000411        time 4m 51s       fid 42.5393   
tick 52    kimg    419.5   loss/reg: G ( 1.034  0.000) D ( 1.072  0.078)   grad norms: G ( 5.990  0.000) D ( 2.318  0.602)   time 1d 00h 05m sec/kimg  181.21 maxGPU  7.4GB clevr-scratch 
tick 53    kimg    427.5   loss/reg: G ( 0.984  0.000) D ( 1.154  0.068)   grad norms: G ( 4.870  0.000) D ( 2.132  0.588

The model has finished training and is currently getting and FID score of 28.95. The weights for this section of the model have been uploaded to our Checkpoints folder on Google Drive. Training or image generation can be resumed by downloading the 604 network snapshot: 

https://drive.google.com/drive/folders/167zK1KXJmceGagUwymjBYiutWbs-7UUi?usp=sharing

-------------------------------------
# Create Dataset for Head CT Scans


In [5]:
# Verify head ct volume mounted in container
!ls head_data | wc -l

170028


In [7]:
!python prepare_data.py \
--task headct \
--images-dir 'head_data' \
--format png --ratio 0.7 \
--max-images 170028 

Preparing the headct dataset...
Loading images from head_data
100%|█████████████████████████████████| 170028/170028 [2:21:20<00:00, 20.05it/s]
Completed preparations for headct!


In [10]:
weights = os.path.join('gansformer', 'results', 'clevr-scratch-000', 'network-snapshot-000604.pkl')

In [23]:
!python run_network.py \
--train \
--gpus 0 \
--ganformer-default \
--expname clevr-scratch \ # Pick up from previous experiment
--dataset headct \
--eval-images-num 10000 \
--keep-samples True \
--metrics fid

Resuming clevr-scratch-000, from results/clevr-scratch-000/network-snapshot-000604.pkl, kimg 604
Local submit - run_dir: results/clevr-scratch-000
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset datasets...
Dataset shape:  [3, 256, 256]
Dynamic range:  [0, 255]
Loading networks from results/clevr-scratch-000/network-snapshot-000604.pkl...
Setting up TensorFlow plugin 'fused_bias_act.cu': Loading... Done.
Setting up TensorFlow plugin 'upfirdn_2d.cu': Loading... Done.

G                                            Params    OutputShape         WeightShape     
---                                          ---       ---                 ---             
ltnt_emb/emb                                 512       (16, 32)            (16, 32)        
G_mapping/AttLayer_0                         6336      (?, 32)             -               
G_mapping/Dense0_0                           1056      (?, 32)             (32, 32)


D                   Params    OutputShape         WeightShape     
---                 ---       ---                 ---             
ltnt_emb/emb        512       (16, 32)            (16, 32)        
256x256/FromRGB     512       (?, 128, 256, 256)  (1, 1, 3, 128)  
256x256/Conv0       147584    (?, 128, 256, 256)  (3, 3, 128, 128)
256x256/Conv1_down  295168    (?, 256, 128, 128)  (3, 3, 128, 256)
256x256/Skip        32768     (?, 256, 128, 128)  (1, 1, 128, 256)
128x128/Conv0       590080    (?, 256, 128, 128)  (3, 3, 256, 256)
128x128/Conv1_down  1180160   (?, 512, 64, 64)    (3, 3, 256, 512)
128x128/Skip        131072    (?, 512, 64, 64)    (1, 1, 256, 512)
64x64/Conv0         2359808   (?, 512, 64, 64)    (3, 3, 512, 512)
64x64/Conv1_down    2359808   (?, 512, 32, 32)    (3, 3, 512, 512)
64x64/Skip          262144    (?, 512, 32, 32)    (1, 1, 512, 512)
32x32/Conv0         2359808   (?, 512, 32, 32)    (3, 3, 512, 512)
32x32/Conv1_down    2359808   (?, 512, 16, 16)    (3, 3, 512,

tick 21    kimg    773.5   loss/reg: G ( 1.353  0.000) D ( 0.895  0.125)   grad norms: G ( 7.998  0.000) D ( 2.252  0.726)   time 9h 05m 28s sec/kimg  179.17 maxGPU  7.5GB clevr-scratch 
network-snapshot-000773        time 4m 53s       fid 49.1943   
tick 22    kimg    781.5   loss/reg: G ( 1.331  0.000) D ( 0.903  0.113)   grad norms: G ( 7.178  0.000) D ( 2.184  0.754)   time 9h 34m 36s sec/kimg  179.29 maxGPU  7.5GB clevr-scratch 
tick 23    kimg    789.6   loss/reg: G ( 1.307  0.000) D ( 0.876  0.110)   grad norms: G ( 6.836  0.000) D ( 2.094  0.742)   time 9h 58m 42s sec/kimg  179.29 maxGPU  7.5GB clevr-scratch 
tick 24    kimg    797.7   loss/reg: G ( 1.367  0.000) D ( 0.843  0.131)   grad norms: G ( 8.036  0.000) D ( 2.234  0.757)   time 10h 22m 57s sec/kimg  180.43 maxGPU  7.5GB clevr-scratch 
network-snapshot-000797        time 4m 52s       fid 44.7494   
tick 25    kimg    805.7   loss/reg: G ( 1.325  0.000) D ( 0.882  0.120)   grad norms: G ( 7.040  0.000) D ( 2.001  0.774) 

tick 49    kimg    999.3   loss/reg: G ( 1.261  0.000) D ( 0.908  0.121)   grad norms: G ( 7.488  0.000) D ( 2.255  1.046)   time 21h 12m 54s sec/kimg  180.20 maxGPU  7.5GB clevr-scratch 
tick 50    kimg   1007.3   loss/reg: G ( 1.260  0.000) D ( 0.899  0.114)   grad norms: G ( 7.450  0.000) D ( 2.162  1.038)   time 21h 37m 11s sec/kimg  180.59 maxGPU  7.5GB clevr-scratch 
tick 51    kimg   1015.4   loss/reg: G ( 1.298  0.000) D ( 0.900  0.106)   grad norms: G ( 7.554  0.000) D ( 2.050  1.038)   time 22h 01m 29s sec/kimg  180.82 maxGPU  7.5GB clevr-scratch 
network-snapshot-001015        time 4m 49s       fid 37.1312   
tick 52    kimg   1023.5   loss/reg: G ( 1.332  0.000) D ( 0.859  0.108)   grad norms: G ( 8.587  0.000) D ( 2.090  1.033)   time 22h 30m 35s sec/kimg  179.52 maxGPU  7.5GB clevr-scratch 
tick 53    kimg   1031.5   loss/reg: G ( 1.387  0.000) D ( 0.856  0.131)   grad norms: G ( 9.331  0.000) D ( 2.227  1.057)   time 22h 54m 46s sec/kimg  179.89 maxGPU  7.5GB clevr-scrat

-------------------
# Generate Samples for Classifier

In [36]:
!python generate.py --gpus 0 --model network-snapshot-001087.pkl --output-dir finals --images-num 1000

Loading networks...
Setting up TensorFlow plugin 'fused_bias_act.cu': Loading... Done.
Setting up TensorFlow plugin 'upfirdn_2d.cu': Loading... Done.

Gs                                           Params    OutputShape         WeightShape     
---                                          ---       ---                 ---             
ltnt_emb/emb                                 512       (16, 32)            (16, 32)        
G_mapping/AttLayer_0                         6336      (?, 32)             -               
G_mapping/Dense0_0                           1056      (?, 32)             (32, 32)        
G_mapping/Dense0_1                           1056      (?, 32)             (32, 32)        
G_mapping/AttLayer_1                         6336      (?, 32)             -               
G_mapping/Dense1_0                           1056      (?, 32)             (32, 32)        
G_mapping/Dense1_1                           1056      (?, 32)             (32, 32)        
G_mapping/AttLayer_2 

100%|██████████| 1000/1000 [01:49<00:00,  9.16image (125 batches of 8 images)/s]
Saving images...
100%|███████████████████████████████████████| 1000/1000 [00:13<00:00, 72.59it/s]
